## Cross sectional regression

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
working_df = pd.read_csv("working_sample_1.csv")

In [3]:
working_df_clean = working_df.drop(columns="Unnamed: 0")
print(working_df_clean.head())

         date  isin  cs_change  delta lr  change_in_10y_bond  change_in_slope  \
0  2022-11-30     0   -0.00267  0.000058            -0.00028         -0.00395   
1  2022-12-31     0   -0.00102  0.000058             0.00517          0.00171   
2  2023-01-31     0   -0.00020  0.000024            -0.00310         -0.00158   
3  2023-02-28     0    0.00077  0.000024             0.00167         -0.00069   
4  2023-03-31     0    0.00259  0.000024            -0.00229         -0.00179   

   smi_return  v3x_change rating  maturity  cpi_change  unempl_change  \
0    0.027314   -0.030361    AAA      10.0      0.0040            0.1   
1   -0.036456   -0.005036    AAA      10.0     -0.2283            0.1   
2    0.050557    0.005429    AAA      10.0      0.6286            0.1   
3   -0.016747   -0.001126    AAA      10.0      0.7838            0.0   
4    0.000710   -0.016202    AAA      10.0      0.1623           -0.1   

   gdp_change  liquidity  credit_rating_numeric     rating_class  \
0    0

### Long-term

In [4]:
df_long_term = working_df_clean[working_df_clean['maturity_bucket'] == "Long-Term"]

print(df_long_term)

             date  isin  cs_change  delta lr  change_in_10y_bond  \
121    2022-07-31     3    0.00094 -0.000117            -0.00584   
122    2022-08-31     3    0.00059 -0.000117             0.00363   
123    2022-09-30     3    0.00201 -0.000117             0.00394   
124    2022-10-31     3   -0.00188 -0.000117            -0.00081   
125    2022-11-30     3   -0.00552 -0.000117            -0.00028   
...           ...   ...        ...       ...                 ...   
36215  2024-08-31   527    0.00001  0.000040             0.00007   
36216  2024-09-30   527    0.00016  0.000040            -0.00054   
36217  2024-10-31   527   -0.00030  0.000040             0.00007   
36218  2024-11-30   527   -0.00011  0.000040            -0.00177   
36219  2024-12-31   527   -0.00010  0.000040             0.00075   

       change_in_slope  smi_return  v3x_change rating  maturity  cpi_change  \
121           -0.00517    0.036985   -0.058189    AAA      25.0     -0.0396   
122           -0.00251   

In [5]:
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

In [6]:
results_by_rating = {}
r_squared_table = {}

coef_table = pd.DataFrame()
for group_name, group_data in df_long_term.groupby('rating_class'):
    y_group = group_data['cs_change']
    x_group = group_data[['delta lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change','cpi_change', 'unempl_change', 'gdp_change','liquidity']]
    x_group = sm.add_constant(x_group)
    model1 = sm.OLS(y_group, x_group).fit()
    
    coef = model1.params
    coef.name = group_name

    coef_table = pd.concat([coef_table, coef], axis=1)
    
    print(f"\n\nResult for {group_name}")
    print(model1.summary)
    print(model1.summary())
    results_by_rating[group_name] = model1
    
    r_squared_table[group_name] = model1.rsquared
    




Result for A class bonds
<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x117542360>>
                            OLS Regression Results                            
Dep. Variable:              cs_change   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     7.351
Date:                Sun, 20 Jul 2025   Prob (F-statistic):           2.01e-10
Time:                        16:09:00   Log-Likelihood:                 5561.4
No. Observations:                 996   AIC:                        -1.110e+04
Df Residuals:                     986   BIC:                        -1.105e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      

In [7]:
# Create R² DataFrame
r_squared_table = pd.Series(r_squared_table, name='R_squared')
# Print
print(r_squared_table)

A class bonds      0.062882
AA class bonds     0.069844
AAA class bonds    0.028082
Not rated bonds    0.069309
Name: R_squared, dtype: float64


### Medium-term

In [8]:
df_medium_term = working_df_clean[working_df_clean['maturity_bucket'] == "Medium-Term"]

print(df_medium_term)

             date  isin  cs_change  delta lr  change_in_10y_bond  \
0      2022-11-30     0   -0.00267  0.000058            -0.00028   
1      2022-12-31     0   -0.00102  0.000058             0.00517   
2      2023-01-31     0   -0.00020  0.000024            -0.00310   
3      2023-02-28     0    0.00077  0.000024             0.00167   
4      2023-03-31     0    0.00259  0.000024            -0.00229   
...           ...   ...        ...       ...                 ...   
36120  2024-08-31   526    0.00158 -0.042684             0.00007   
36121  2024-09-30   526    0.00052 -0.042684            -0.00054   
36122  2024-10-31   526   -0.00631 -0.042684             0.00007   
36123  2024-11-30   526   -0.00187 -0.042684            -0.00177   
36124  2024-12-31   526    0.00341 -0.042684             0.00075   

       change_in_slope  smi_return  v3x_change rating  maturity  cpi_change  \
0             -0.00395    0.027314   -0.030361    AAA      10.0      0.0040   
1              0.00171   

In [9]:
results_by_rating = {}
r_squared_table = {}

coef_table = pd.DataFrame()
for group_name, group_data in df_medium_term.groupby('rating_class'):
    y_group = group_data['cs_change']
    x_group = group_data[['delta lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change', 'cpi_change', 'unempl_change', 'gdp_change','liquidity']]
    x_group = sm.add_constant(x_group)
    model1 = sm.OLS(y_group, x_group).fit()
    
    coef = model1.params
    coef.name = group_name

    coef_table = pd.concat([coef_table, coef], axis=1)
    
    print(f"\n\nResult for {group_name}")
    print(model1.summary)
    print(model1.summary())
    results_by_rating[group_name] = model1
    
    r_squared_table[group_name] = model1.rsquared
    

coef_table = coef_table.T



Result for A class bonds
<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x116415640>>
                            OLS Regression Results                            
Dep. Variable:              cs_change   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     25.05
Date:                Sun, 20 Jul 2025   Prob (F-statistic):           4.52e-42
Time:                        16:09:00   Log-Likelihood:                 17786.
No. Observations:                3352   AIC:                        -3.555e+04
Df Residuals:                    3342   BIC:                        -3.549e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      

In [10]:
# Create R² DataFrame
r_squared_table = pd.Series(r_squared_table, name='R_squared')
# Print
print(r_squared_table)

A class bonds      0.063199
AA class bonds     0.043587
AAA class bonds    0.064526
BBB/BB/B bonds     0.010211
Not rated bonds    0.017550
Name: R_squared, dtype: float64


### Short-term

In [11]:
df_short_term = working_df_clean[working_df_clean['maturity_bucket'] == "Short-Term"]

print(df_short_term.head())

            date  isin  cs_change  delta lr  change_in_10y_bond  \
1060  2021-05-31    18    0.00009 -0.000107             0.00045   
1061  2021-06-30    18    0.00009 -0.000107            -0.00069   
1062  2021-07-31    18   -0.00024 -0.000107            -0.00135   
1063  2021-08-31    18    0.00024 -0.000107             0.00043   
1064  2021-09-30    18    0.00083 -0.000107             0.00145   

      change_in_slope  smi_return  v3x_change rating  maturity  cpi_change  \
1060          0.00041    0.035981   -0.008706   BBB+       3.0      0.2664   
1061         -0.00054    0.044217   -0.001625   BBB+       3.0      0.0748   
1062         -0.00140    0.014473    0.003346   BBB+       3.0     -0.0857   
1063          0.00078    0.023998   -0.004311   BBB+       3.0      0.2481   
1064          0.00087   -0.063935    0.038714   BBB+       3.0      0.0176   

      unempl_change  gdp_change  liquidity  credit_rating_numeric  \
1060           -0.3    0.464417       0.73                 

In [12]:
results_by_rating = {}
r_squared_table = {}

coef_table = pd.DataFrame()
for group_name, group_data in df_short_term.groupby('rating_class'):
    y_group = group_data['cs_change']
    x_group = group_data[['delta lr', 'change_in_10y_bond', 'change_in_slope', 'smi_return', 'v3x_change','cpi_change', 'unempl_change', 'gdp_change','liquidity']]
    x_group = sm.add_constant(x_group)
    model1 = sm.OLS(y_group, x_group).fit()
    
    coef = model1.params
    coef.name = group_name

    coef_table = pd.concat([coef_table, coef], axis=1)
    
    print(f"\n\nResult for {group_name}")
    print(model1.summary)
    print(model1.summary())
    results_by_rating[group_name] = model1
    
    r_squared_table[group_name] = model1.rsquared
    

coef_table = coef_table.T



Result for A class bonds
<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x117542360>>
                            OLS Regression Results                            
Dep. Variable:              cs_change   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     2.753
Date:                Sun, 20 Jul 2025   Prob (F-statistic):            0.00669
Time:                        16:09:00   Log-Likelihood:                 510.64
No. Observations:                 104   AIC:                            -1001.
Df Residuals:                      94   BIC:                            -974.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      

In [13]:
# Create R² DataFrame
r_squared_table = pd.Series(r_squared_table, name='R_squared')
# Print
print(r_squared_table)

A class bonds      0.208628
AA class bonds     0.132566
AAA class bonds    0.102762
BBB/BB/B bonds     0.100911
Not rated bonds    0.077863
Name: R_squared, dtype: float64
